In [1]:
from pcd_dataset.kitti_dataset import KittiDataset

dataset_path = 'dataset_baby/'
sequence = '00'
image_instances_path = 'pipeline_baby/vfm-labels/sam/00/'
kitti = KittiDataset(dataset_path, sequence, image_instances_path)

In [2]:
start_index = 0
end_index = 3
cam_name = 'cam2'

In [3]:
from services.label_service import get_map_not_zero_in_sphere

map, points2instances = get_map_not_zero_in_sphere(kitti, cam_name, start_index, end_index, 10, True)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [4]:
from services.label_service import build_custom_voxel_pcd

voxel_pcd, voxel_points2instances, voxel_trace = build_custom_voxel_pcd(map, points2instances)

# import open3d as o3d
# from utils.pcd_utils import color_pcd_by_labels

# voxel_pcd_colored = color_pcd_by_labels(voxel_pcd, voxel_points2instances[:, 1])
# o3d.visualization.draw_geometries([voxel_pcd_colored])

In [7]:
# counter1 = 0
# counter2 = 0
# import numpy as np
# list_int_vectors123 = voxel_trace[2]
# for i in range(len(list_int_vectors123)):
#     int_vector_array = np.asarray(list_int_vectors123[i])
#     if (int_vector_array == [2, 0, 0]).all():
#         counter2 += 1
#     for j in range(len(int_vector_array)):
#         if int_vector_array[j] == 2:
#             counter1 += 1

# print(counter1)
# print(counter2)

ValueError: operands could not be broadcast together with shapes (2,) (3,) 

In [13]:
voxel_trace[2][5220]

IntVector[1623, 1624, 4684, 4685, 9129, 9130]

In [14]:
# с этим примером все супер, тут реально вычисляются середины
print(voxel_trace[2][5220])
print(points2instances[1623])
print(points2instances[1624])
print(points2instances[4684])
print(points2instances[4685])
print(points2instances[9129])
print(points2instances[9130])

print(map.points[1623])
print(map.points[1624])
print(map.points[4684])
print(map.points[4685])
print(map.points[9129])
print(map.points[9130])

print("voxel={}".format(voxel_trace[0].points[5220]))

IntVector[1623, 1624, 4684, 4685, 9129, 9130]
[2 0 0]
[2 0 0]
[2 0 0]
[2 0 0]
[2 0 0]
[2 0 0]
[23.0776773   0.04443617 -1.46711334]
[23.07989703  0.06659685 -1.46724863]
[23.08430135  0.05226152 -1.46585838]
[23.08764286  0.07148429 -1.46696513]
[23.10217278  0.04884453 -1.47280196]
[23.08656071  0.06693017 -1.46699748]
voxel=[23.08637534  0.05842559 -1.46783082]


In [28]:
# все супер, просто усреднили все координаты
print(voxel_trace[2][1367])
print(points2instances[1621])
print(points2instances[1622])
print(points2instances[4680])
print(points2instances[4681])
print(points2instances[9126])
print(points2instances[9127])

print(map.points[1621])
print(map.points[1622])
print(map.points[4680])
print(map.points[4681])
print(map.points[9126])
print(map.points[9127])

value = map.points[1621][1] + map.points[1622][1] + map.points[4680][1] + map.points[4681][1] + map.points[9126][1] + map.points[9127][1]
print("valeu={}".format(value/6))

print("voxel={}".format(voxel_trace[0].points[1367]))

IntVector[1621, 1622, 4680, 4681, 9126, 9127]
[2 0 0]
[2 0 0]
[2 0 0]
[2 0 0]
[2 0 0]
[2 0 0]
[ 2.30912227e+01 -1.18445522e-02 -1.47164951e+00]
[ 2.30934443e+01  1.03161256e-02 -1.47178480e+00]
[ 2.30871740e+01 -1.56681853e-02 -1.46840845e+00]
[ 2.30923672e+01 -6.34413795e-03 -1.46944261e+00]
[ 2.31003847e+01 -9.35678853e-03 -1.47441343e+00]
[ 2.31016592e+01 -1.26641592e-03 -1.47446266e+00]
valeu=-0.005693992381612241
voxel=[ 2.30943753e+01 -5.69399238e-03 -1.47169358e+00]


In [21]:
from services.label_service import build_custom_voxel_pcd

voxel_pcd1, voxel_points2instances1, voxel_trace1 = build_custom_voxel_pcd(map, points2instances, 0.1)

4
9
12
13
16
17
18
19
24
28
37
39
43
48
50
51
52
53
54
55
60
61
65
68
69
75
78
82
83
94
97
126
195
197
200
204
209
215
217
228
240
241
256
258
263
266
279
281
282
288
291
298
303
312
325
336
339
340
341
351
372
386
398
401
405
421
429
430
431
439
443
444
450
463
474
475
476
497
510
527
529
545
579
585
587
598
599
604
615
627
636
637
648
661
687
699
725
732
737
747
767
784
787
795
814
828
866
870
874
882
905
906
914
934
937
952
968
971
982
1002
1006
1016
1025
1065
1081
1092
1101
1115
1121
1132
1146
1147
1157
1159
1174
1189
1195
1201
1213
1214
1225
1226
1237
1266
1273
1296
1298
1314
1318
1338
1356
1359
1368
1374
1376
1377
1383
1396
1399
1413
1436
1443
1446
1461
1465
1469
1473
1493
1502
1515
1516


In [23]:
import numpy as np

list_int_vectors123 = voxel_trace1[2]

counter1 = 0
counter2 = 0
for i in range(len(list_int_vectors123)):
    int_vector_array = np.asarray(list_int_vectors123[i])
    if int_vector_array == [2, 0, 0]
    for j in range(len(int_vector_array)):
        if int_vector_array[j] == 2:
            counter1 += 1

print(counter1)

33


In [24]:
import numpy as np

list_int_vectors123 = voxel_trace1[2]

maxx = 0
for i in range(len(list_int_vectors123)):
    int_vector_array = np.asarray(list_int_vectors123[i])
    if len(int_vector_array) == 33:
        print(i)

1427


In [29]:
arr

array([ 137,  138,  139,  143,  144, 2110, 2111, 2112, 2118, 2119, 2120,
       2122, 2130, 2131, 2133, 2134, 2135, 5327, 5328, 5329, 5330, 5331,
       5332, 5333, 5340, 5341, 5342, 5343, 5344, 5345, 5346, 5358, 5359],
      dtype=int32)

In [27]:
# привести как пример что надо проверять инстансы
arr = np.asarray(list_int_vectors123[1427])
for i in range(len(arr)):
    print("i={}, inst={}".format(i, points2instances[i]))


i=0, inst=[14 13 33]
i=1, inst=[14 13 33]
i=2, inst=[14 13 33]
i=3, inst=[14 13 33]
i=4, inst=[14 13 33]
i=5, inst=[14 13 33]
i=6, inst=[14 13 33]
i=7, inst=[0 0 5]
i=8, inst=[14 13 33]
i=9, inst=[14 13 33]
i=10, inst=[14 13 33]
i=11, inst=[14 13 33]
i=12, inst=[14 13 33]
i=13, inst=[14 13 33]
i=14, inst=[14 13 33]
i=15, inst=[14  0 33]
i=16, inst=[14 13 33]
i=17, inst=[14 13 33]
i=18, inst=[14 13 33]
i=19, inst=[14 13 33]
i=20, inst=[14 13 33]
i=21, inst=[14 13 33]
i=22, inst=[14 13 18]
i=23, inst=[14 13 33]
i=24, inst=[14 13 33]
i=25, inst=[14 13 33]
i=26, inst=[14 13 33]
i=27, inst=[14 13 33]
i=28, inst=[14 13 33]
i=29, inst=[14 13 18]
i=30, inst=[14 13 18]
i=31, inst=[14 13 18]
i=32, inst=[14 13 18]


In [22]:
len(voxel_trace1[0].points)

1529

In [5]:
import numpy as np

from scipy.spatial.distance import cdist
from utils.distances_utils import sam_label_distance

points = np.asarray(voxel_pcd.points)
spatial_distance = cdist(points, points)

print("len(voxel_pcd.points) = {}".format(len(voxel_pcd.points)))
print("len(voxel_points2instances) = {}".format(len(voxel_points2instances)))

dist, masks = sam_label_distance(voxel_points2instances, spatial_distance, 2, 10)

len(voxel_pcd.points) = 13490
len(voxel_points2instances) = 13490


In [6]:
from services.normalized_cut_service import normalized_cut

T = 0.3
clusters = normalized_cut(dist, np.asarray(points), T)

/Library/Python/3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 11058 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)
/Users/sofiavivdich/proj/lidar-labelling/services/normalized_cut_service.py:27: RuntimeWarning: invalid value encountered in scalar divide
  return (cost / assoc_a) + (cost / assoc_b)


In [7]:
len(clusters)

2

In [10]:
import numpy as np
from scipy.spatial import KDTree

cloud1 = np.asarray(map.points)
cloud2 = np.asarray(voxel_pcd.points)

# Build KD-trees for each point cloud
tree1 = KDTree(cloud1)
tree2 = KDTree(cloud2)

# Find nearest neighbors for each point in cloud1
dists, indices = tree2.query(cloud1, k=1)

# Get the neighbor points from cloud2
neighbors = cloud2[indices]

In [11]:
from utils.pcd_utils import color_pcd_by_clusters_common

import open3d as o3d

pcd_clustered1, pcd_clustered2, counter1, counter2 = color_pcd_by_clusters_common(map, clusters, neighbors)

print("counter1 = {}, counter2 = {}".format(counter1, counter2))
print("removed points = {}".format(len(map.points) - len(voxel_pcd.points)))
print("len(map.points) = {}".format(len(map.points)))
print("len(voxel_pcd.points) = {}".format(len(voxel_pcd.points)))

o3d.visualization.draw_geometries([pcd_clustered1])
o3d.visualization.draw_geometries([pcd_clustered2])

In [31]:
import open3d as o3d

pcd = o3d.io.read_point_cloud("clouds_20_25_im21.pcd")
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
